#### Information Storage and Retrieval :: Link Analysis

# Topic:  PageRank + Learning to Rank


*Goals of this Tutorial:* In this work, we will explore real-world challenges of building a graph from tweets, implement and test the classic PageRank algortihm over this graph. In addition, a learning to rank scheme is applied to a real-world dataset and report the performance in terms of NDCG.


# Part 1: PageRank
In Part 1, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. 

Christopher D. Manning, Prabhakar Raghavan, and Hinrich Schutze, "Introduction to Information Retrieval"
<br>[PAGERANK](https://nlp.stanford.edu/IR-book/pdf/21link.pdf) (Chap 21.2)


## Part 1.1: A re-Tweet Graph

So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is retweeted by other users is more "impactful". 

Here is a toy example. Suppose the following four retweets are given:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *PageRank.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are binary and directed. If Bob retweets Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [0]:
# Here define your function for building the graph by parsing 
# the input file of tweets
# Insert as many cells as you want
import json
from collections import defaultdict

# id of the user in the retweeted_status
def Build_Graph(InFile, outedges, inedges):
    with open (InFile, encoding='utf-8', mode = 'r') as f:
        for line in f:
            data = json.loads(line)
            source_id = data["user"]["id"]
            destination_id = data["retweeted_status"]["user"]["id"]

            # If a user retweets herself, ignore it.
            if source_id == destination_id:
                continue

            # Initialize outedges, inedges for source_id, destination_id
            if destination_id not in outedges:
                outedges[destination_id] = set()
            if source_id not in outedges:
                outedges[source_id] = set()
            if destination_id not in inedges:
                inedges[destination_id] = set()
            if source_id not in inedges:
                inedges[source_id] = set()
    
            # Add value to key
            outedges[source_id].add(destination_id)
            inedges[destination_id].add(source_id)

    return (outedges, inedges)

def Calculate_Graph_Size(num_of_nodes, num_of_edges):
    num_of_nodes = len(outedges)
    for node in outedges.keys():
        num_of_edges += len(outedges[node])

    # Double Check the correctness of Graph
    if len(outedges) == len(inedges):
      print ("Number of nodes is Matched")
    else:
      print ("The calculation is Error")
    return (num_of_nodes, num_of_edges)

In [0]:
# Call your function to print out the size of the graph, 
# i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you 
#write the graph into a file, and load it later.

inedges = defaultdict(set)
outedges = defaultdict(set)
outedges = defaultdict(set)
num_of_nodes = 0
num_of_edges = 0
outedges, inedges = Build_Graph("HITS.json", outedges, inedges)
num_of_nodes, num_of_edges = Calculate_Graph_Size(num_of_nodes, num_of_edges)
print ("num of nodes : " + str(num_of_nodes))
print ("num of edges : " + str(num_of_edges))


Number of nodes is Matched
num of nodes : 1003
num of edges : 6177


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## Part 1.2: PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never retweeted and does not retweet anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

*ADD YOUR ANSWER HERE:*
**After each iteration, my script will acquire pagerank values for each user id.**
**Therfore, I set the termination condition by comparing the the pagerank absolute difference between the current and previous iteration. If the absolute difference is larger than epsilon(2.220446049250313e-16), then the iteration keep ruunning, or stop on the other hand.**

In [0]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want
# def PageRanker(...):
#    ...
import copy
import operator
import math
import sys
def PageRanker(outedges, inedges, num_of_nodes, k):
    # Initialization of starting state
    cur = []
    prev = []
    for node in outedges.keys():
        page_rank[node] = 1.0 / num_of_nodes
    iterations = 0

    # Iterative Calculations for PageRank
    while True:
        iterations = iterations +1
        tmp_pagerank = copy.deepcopy(page_rank)
        for node in outedges.keys():
            page_rank[node] = 0.0
            for in_node in inedges[node]:
                page_rank[node] = page_rank[node] + tmp_pagerank[in_node] / len(outedges[in_node])
            page_rank[node] = d * page_rank[node] + (1 - d) / num_of_nodes
        cur = sorted(page_rank.items(), key = operator.itemgetter(1), reverse=True)

        # Check Convergence pagerank results
        count = num_of_nodes
        if is_converged(cur, prev, count) == True:
            break
        else:
            prev = cur
    print ("num_of_iterations : " + str(iterations))

    # Normalization of page_rank scores
    total_pagerank = sum(iter(page_rank.values()) )
    for key, value in page_rank.items():
        page_rank[key] = float(value)/total_pagerank
    
    # List out the top k results with higher page_rank
    rank = 0
    for w in sorted(page_rank, key=page_rank.get, reverse=True):
        if rank == k:
            break
        print (w, "-", page_rank[w])
        rank += 1

def is_converged(cur, prev, count):
    if prev == []:
        return False
    count = 10
    for i in range(count):
        if abs(cur[i][0] - prev[i][0]) > sys.float_info.epsilon:
            return False
    return True

In [0]:
# Now let's call your function on the graph you've built. Output the results.
page_rank = {}
d = 0.9 # the damping factor is 0.9
PageRanker(outedges, inedges, num_of_nodes, 10)


num_of_iterations : 9
1183906148 - 0.02982668510041511
3019659587 - 0.02202843142696112
3077695572 - 0.02152795498447896
3068694151 - 0.019102526748325743
2598548166 - 0.018943885068987927
3042570996 - 0.01803456919473446
3154266823 - 0.017921533583318754
571198546 - 0.017164772381839763
3082766914 - 0.015847616166215214
3039321886 - 0.015356551257285515


## Part 1.3: Improving PageRank
In the many years since PageRank was introduced, there have been many improvements and extensions. For this part, you should experiment with one such improvement and then compare the results you get with the original results in Part 1.2. 

In [0]:
# Here add your code
import json
from collections import defaultdict
import copy
import operator
import math
import sys

# id of the user
# id of the user in the retweeted_status
# screen_name of the user in the retweeted_status
def Build_Graph_Consider_Weights(InFile, outedges, inedges, inweights, LOG_weight=False):
    with open (InFile, encoding='utf-8', mode = 'r') as f:
        for line in f:
            data = json.loads(line)
            source_id = data["user"]["id"]
            destination_id = data["retweeted_status"]["user"]["id"]
            retweet_count = data["retweeted_status"]["retweet_count"]

            # If a user retweets herself, ignore it.
            if source_id == destination_id:
                continue

            # Initialize outedges, inedges for source_id, destination_id
            if destination_id not in outedges:
                outedges[destination_id] = set()
            if source_id not in outedges:
                outedges[source_id] = set()
            if destination_id not in inedges:
                inedges[destination_id] = set()
            if source_id not in inedges:
                inedges[source_id] = set()
            if (source_id, destination_id) not in inweights:
                inweights[(source_id, destination_id)] = 0
    
            # Add value to key
            outedges[source_id].add(destination_id)
            inedges[destination_id].add(source_id)
            inweights[(source_id, destination_id)] = inweights[(source_id, destination_id)] + retweet_count
        if LOG_weight == True:
            for source_id, destination_id in inweights:
                if inweights[(source_id, destination_id)] == 0:
                    inweights[(source_id, destination_id)] = 0
                else:
                  inweights[(source_id, destination_id)] = 1 + math.log2(inweights[(source_id, destination_id)])

    return (outedges, inedges, inweights)

def PageRanker_Consider_Weights(outedges, inedges, num_of_nodes, k, inweights, page_rank):
    # Initialization of starting state
    cur = []
    prev = []
    for node in outedges.keys():
        page_rank[node] = 1.0 / num_of_nodes
    iterations = 0

    # Iterative Calculations for PageRank
    while True:
        iterations = iterations +1
        tmp_pagerank = copy.deepcopy(page_rank)
        for node in outedges.keys():
            page_rank[node] = 0.0
            # Calculate the total inweights on node inweights[(source_id, destination_id)]
            totalweight = 0.0
            for in_node in inedges[node]:
                totalweight = totalweight + inweights[(in_node, node)]

            for in_node in inedges[node]:
                #page_rank[node] = page_rank[node] + tmp_pagerank[in_node] / len(outedges[in_node])
                if totalweight == 0:
                    break
                page_rank[node] = page_rank[node] + tmp_pagerank[in_node]*inweights[(in_node, node)] / totalweight
            # No Random walk to itself
            if in_node == node:
                  continue
            page_rank[node] = d * page_rank[node] + (1 - d) / num_of_nodes
            # Normalization of page_rank scores
            total_pagerank = sum(iter(page_rank.values()) )
            for key, value in page_rank.items():
                page_rank[key] = float(value)/total_pagerank
        cur = sorted(page_rank.items(), key = operator.itemgetter(1), reverse=True)

        count = num_of_nodes
        if is_converged(cur, prev, count) == True:
            break
        else:
            prev = cur
    print ("num_of_iterations : " + str(iterations))

    # Normalization of page_rank scores
    total_pagerank = sum(iter(page_rank.values()) )
    for key, value in page_rank.items():
        page_rank[key] = float(value)/total_pagerank
    
    # List out the top k results with higher page_rank
    rank = 0
    for w in sorted(page_rank, key=page_rank.get, reverse=True):
        if rank == k:
            break
        print (w, "-", page_rank[w])
        rank += 1

def is_converged(cur, prev, count):
    if prev == []:
        return False
    for i in range(count):
        if abs(cur[i][0] - prev[i][0]) > sys.float_info.epsilon:
            return False
    return True

def Calculate_Graph_Size(num_of_nodes, num_of_edges):
    num_of_nodes = len(outedges)
    for node in outedges.keys():
        num_of_edges += len(outedges[node])
    # Double Check the correctness of Graph
    #if len(outedges) == len(inedges):
    #  print ("Number of nodes is Matched")
    #else:
    #  print ("The calculation is Error")
    return (num_of_nodes, num_of_edges)

inedges = defaultdict(set)
outedges = defaultdict(set)
inweights = defaultdict()
num_of_nodes = 0
num_of_edges = 0
outedges, inedges, inweights = Build_Graph_Consider_Weights("HITS.json", outedges, inedges, inweights)
num_of_nodes, num_of_edges = Calculate_Graph_Size(num_of_nodes, num_of_edges)
print ("==== Considering the retweet_count as the preference of users ====")
page_rank = {}
d = 0.9 # the damping factor is 0.9
PageRanker_Consider_Weights(outedges, inedges, num_of_nodes, 10, inweights, page_rank)

print ("==== Transforming the retweet_count to LOG-based weights ====")
outedges, inedges, inweights = Build_Graph_Consider_Weights("HITS.json", outedges, inedges, inweights, LOG_weight=True)
page_rank_log_weights = {}
PageRanker_Consider_Weights(outedges, inedges, num_of_nodes, 10, inweights, page_rank_log_weights)


==== Considering the retweet_count as the preference of users ====
num_of_iterations : 144
3097303928 - 0.057043546753658525
3092183276 - 0.049798751508822
3096128614 - 0.03985988055715409
3105280887 - 0.02802991970102105
3043706996 - 0.02497497143572098
374650547 - 0.020907872383452087
1616036438 - 0.018961123249702456
2985751937 - 0.01795516827392531
3096831739 - 0.017101058653058637
3016832732 - 0.016318845359938616
==== Transforming the retweet_count to LOG-based weights ====
num_of_iterations : 56
3097303928 - 0.006321934383023622
3167550245 - 0.005119402820780373
2777167148 - 0.004799814002560403
3092183276 - 0.0046219499819348826
2813168973 - 0.004514463752361885
1616036438 - 0.004294244487552252
3158081854 - 0.004247476733649387
3158566838 - 0.004192676749082602
3161738230 - 0.0041859030070623094
2505878780 - 0.004180270131428651


In [0]:
# Plus be sure to describe your extension (what is it? 
# why did you choose it?) and your comparison to Part 1.2

## [ANS] ##
Compared with Part 1.2 that adopts binary value on edge weights,
I introduce the integer weights to all edges in Part 1.3, for instance, the edge weight is 10
when the text is retweeted 10 times, which means the text is more important for this user.

The reason is that I think we should consider the preference of users retweet message,
not just use 0 or 1 to analyze the link analysis.

Based on this property, I provided another revised improvement: LOG-based weight values.
In order to discount the domoniance of specific strong weight.
If the original_weight == 0: then the LOG-based weight = 0
Eles: then the LOG-based weight = 1 + log2(original_weight)

# Part 2: Learning to Rank

For this part, we're going to play with some Microsoft LETOR data that has query-document relevance judgments. Let's see how learning to rank works in practice. 

First, you will need to download the MQ2008.zip file from the Resources tab on Piazza. This is data from the [Microsoft Research IR Group](https://www.microsoft.com/en-us/research/project/letor-learning-rank-information-retrieval/).

The data includes 15,211 rows. Each row is a query-document pair. The first column is a relevance label of this pair (0,1 or 2--> the higher value the more related), the second column is query id, the following columns are features, and the end of the row is comment about the pair, including id of the document. A query-document pair is represented by a 46-dimensional feature vector. Features are a numeric value describing a document and query such as TFIDF, BM25, Page Rank, .... You can find compelete description of features from [here](https://arxiv.org/ftp/arxiv/papers/1306/1306.2597.pdf).

The dataset is ready for analysis: It has already been split into 5 folds (see the five folders called Fold1, ..., Fold5).

Tie-Yan Liu, "[Learning to Rank for Information Retrieval](https://www.nowpublishers.com/article/Details/INR-016)", , Microsoft Research Asia, Sigma Center



## Part 2.1: Build Point-wise Learning to Rank
First, you should build a point-wise Learning to Rank framework. 
1. You could train a binary classification model like SVM or logistic regression on the train file. In this case, 0 is treated as negative (irrelevant) sample and 1, 2 are treated as positive (relevant) sample.
2. You apply the already trained model to predict the scores for documents on test file.
3. Order the documents based on the scores.

In [0]:
import numpy as np
from sklearn.datasets import load_svmlight_file
import scipy.sparse as sp
from collections import defaultdict
num_of_features = 46

def Parse_Data(Folder):
    # Parse data from Fold/ and transform y from {0,1,2} to {0,1}
    x_train1, y_train1 = load_svmlight_file(str(Folder) + "/train.txt", n_features=num_of_features)
    x_valid1, y_valid1 = load_svmlight_file(str(Folder) + "/vali.txt", n_features=num_of_features)
    x_test1, y_test1 = load_svmlight_file(str(Folder) + "/test.txt", n_features=num_of_features)
    for i in range(len(y_train1)):
        if y_train1[i] > 0:
            y_train1[i] = 1
    for i in range(len(y_valid1)):
        if y_valid1[i] > 0:
            y_valid1[i] = 1
    for i in range(len(y_test1)):
        if y_test1[i] > 0:
            y_test1[i] = 1
    return (x_train1, x_valid1, x_test1, y_train1, y_valid1, y_test1)

from sklearn import svm
from sklearn.model_selection import GridSearchCV

def Cross_Validation_And_Score (x_train, y_train, x_valid, y_valid):
    Cs = [0.5, 0.8, 1, 1.2, 10]
    gammas = [0.5, 0.9, 1, 10, 'scale', 'auto']
    param_grid = {'C': Cs, 'gamma' : gammas}
    clf = svm.SVC(C=1, gamma=1, kernel='rbf', probability=True)
    clf.fit(x_train, y_train)
    grid = GridSearchCV(clf, param_grid, refit=True, cv=2)
    grid.fit(x_valid, y_valid)
    print(grid.best_estimator_)

from joblib import dump, load
############# Fold1, score: 0.7957148134466199 #############
(x_train1, x_valid1, x_test1, y_train1, y_valid1, y_test1) = Parse_Data("Fold1")
print ("=== HyperParameters after validations of Fold1 ===")
Cross_Validation_And_Score (x_train1, y_train1, x_valid1, y_valid1)
# C=10, gamma='auto', kernel='rbf' (best hyperparameters of model_1)

############# Fold2, score: 0.8068893528183716 #############
(x_train2, x_valid2, x_test2, y_train2, y_valid2, y_test2) = Parse_Data("Fold2")
print ("=== HyperParameters after validations of Fold2 ===")
Cross_Validation_And_Score (x_train2, y_train2, x_valid2, y_valid2)
# C=0.5, gamma='auto', kernel='rbf' (best hyperparameters of model_2)

############# Fold3, score: 0.7933856120013638 #############
(x_train3, x_valid3, x_test3, y_train3, y_valid3, y_test3) = Parse_Data("Fold3")
print ("=== HyperParameters after validations of Fold3 ===")
Cross_Validation_And_Score (x_train3, y_train3, x_valid3, y_valid3)
# C=0.8, gamma=0.9, kernel='rbf' (best hyperparameters of model_3)

############# Fold4, score: 0.8437414030261348 #############
(x_train4, x_valid4, x_test4, y_train4, y_valid4, y_test4) = Parse_Data("Fold4")
print ("=== HyperParameters after validations of Fold4 ===")
Cross_Validation_And_Score (x_train4, y_train4, x_valid4, y_valid4)
# C=1, gamma=0.5, kernel='rbf' (best hyperparameters of model_4)

############# Fold5, score: 0.7916394513389942 #############
(x_train5, x_valid5, x_test5, y_train5, y_valid5, y_test5) = Parse_Data("Fold5")
print ("=== HyperParameters after validations of Fold5 ===")
Cross_Validation_And_Score (x_train5, y_train5, x_valid5, y_valid5)
# C=1.2, gamma='auto', kernel='rbf' (best hyperparameters of model_5)
print("SVM model 4, clf4, has the hyperparameter C=1, gamma=0.5, kernel='rbf' in validation stage")

############ Retrain the test based on clf4's HyperParameters ############
############ C=1, gamma=0.5, kernel='rbf' ############
clf1 = svm.SVC(C=1, gamma=0.5, kernel='rbf', probability=True)
clf1.fit(x_train1, y_train1)
print ("clf1 score: " + str(clf1.score (x_valid1, y_valid1)))
dump(clf1, 'clf1.joblib') 

clf2 = svm.SVC(C=1, gamma=0.5, kernel='rbf', probability=True)
clf2.fit(x_train2, y_train2)
print ("clf2 score: " + str(clf2.score (x_valid2, y_valid2)))
dump(clf2, 'clf2.joblib') 

clf3 = svm.SVC(C=1, gamma=0.5, kernel='rbf', probability=True)
clf3.fit(x_train3, y_train3)
print ("clf3 score: " + str(clf3.score (x_valid3, y_valid3)))
dump(clf3, 'clf3.joblib') 

clf4 = svm.SVC(C=1, gamma=0.5, kernel='rbf', probability=True)
clf4.fit(x_train4, y_train4)
print ("clf4 score: " + str(clf4.score (x_valid4, y_valid4)))
dump(clf4, 'clf4.joblib') 

clf5 = svm.SVC(C=1.2, gamma='auto', kernel='rbf', probability=True)
clf5.fit(x_train5, y_train5)
print ("clf5 score: " + str(clf5.score (x_valid5, y_valid5)))
dump(clf5, 'clf5.joblib') 

=== HyperParameters after validations of Fold1 ===
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
=== HyperParameters after validations of Fold2 ===
SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
=== HyperParameters after validations of Fold3 ===
SVC(C=0.8, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.9, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
=== HyperParameters after validations of Fold4 ===
SVC(C=1, break_ties=False, cache_size=200, class_weight=

['clf5.joblib']

In [0]:
##### Ranking the (q,d) pair #####
import numpy as np
from sklearn.datasets import load_svmlight_file
from collections import defaultdict
num_of_features = 46

def Parse_Test_Data (Folder):
    x_test, y_test, q_id = load_svmlight_file(str(Folder) + "/test.txt", query_id = True, n_features=num_of_features)
    outFile = open(str(Folder) + "/test.txt", 'r', encoding="utf-8")
    doc_id = []
    for line in outFile:
        doc_id.append(line.split('#docid = ', 1)[-1].split(' ', 1)[0])
    return (x_test, y_test, q_id, doc_id)

def Rank_by_Log_Proba (x_test, y_test, q_id, doc_id, clf):
    c = clf.predict_log_proba (x_test)
    log_prob = c[:,1]
    unique_query = np.unique(q_id)
    Ranked_of_q_doc = defaultdict(list)

    idx = 0
    for item in q_id:
        Ranked_of_q_doc[str(item)].append([ doc_id[idx], log_prob[idx], y_test[idx] ])
        idx = idx+1
    # Sort the (q,d) based on predict_log_proba results
    for item in q_id:
        tmp = Ranked_of_q_doc[str(item)]
        tmp = sorted(tmp, key = lambda x: x[1], reverse = True)
        Ranked_of_q_doc[str(item)] = tmp

    Ground_Truth_q_doc = defaultdict(list)
    # Sort the (q,d) based on Ground Truth: y_test
    for item in q_id:
        tmp = Ranked_of_q_doc[str(item)]
        tmp = sorted(tmp, key = lambda x: x[2], reverse = True)
        Ground_Truth_q_doc[str(item)] = tmp

    return (unique_query, Ranked_of_q_doc, Ground_Truth_q_doc)

################# Load the Trained Models, and rank (q,d) based on predict_log_proba ######################
from joblib import dump, load
clf1 = load('clf1.joblib') 
clf2 = load('clf2.joblib') 
clf3 = load('clf3.joblib')
clf4 = load('clf4.joblib') 
clf5 = load('clf5.joblib')
(x_test1, y_test1, q_id1, doc_id1) = Parse_Test_Data("Fold1")
(x_test2, y_test2, q_id2, doc_id2) = Parse_Test_Data("Fold2")
(x_test3, y_test3, q_id3, doc_id3) = Parse_Test_Data("Fold3")
(x_test4, y_test4, q_id4, doc_id4) = Parse_Test_Data("Fold4")
(x_test5, y_test5, q_id5, doc_id5) = Parse_Test_Data("Fold5")

print ("=== Rank 5 test data based on predict_log_proba value of 5 trained-validated-models ===")
(unique_query1, Ranked_of_q_doc1, Ground_Truth_q_doc1) = Rank_by_Log_Proba (x_test1, y_test1, q_id1, doc_id1, clf1)
print (Ranked_of_q_doc1)
(unique_query2, Ranked_of_q_doc2, Ground_Truth_q_doc2) = Rank_by_Log_Proba (x_test2, y_test2, q_id2, doc_id2, clf2)
print (Ranked_of_q_doc2)
(unique_query3, Ranked_of_q_doc3, Ground_Truth_q_doc3) = Rank_by_Log_Proba (x_test3, y_test3, q_id3, doc_id3, clf3)
print (Ranked_of_q_doc3)
(unique_query4, Ranked_of_q_doc4, Ground_Truth_q_doc4) = Rank_by_Log_Proba (x_test4, y_test4, q_id4, doc_id4, clf4)
print (Ranked_of_q_doc4)
(unique_query5, Ranked_of_q_doc5, Ground_Truth_q_doc5) = Rank_by_Log_Proba (x_test5, y_test5, q_id5, doc_id5, clf5)
print (Ranked_of_q_doc5)
print ("=== Ground Truth Ranking ===")
print (Ground_Truth_q_doc1)
print (Ground_Truth_q_doc2)
print (Ground_Truth_q_doc3)
print (Ground_Truth_q_doc4)
print (Ground_Truth_q_doc5)

=== Rank 5 test data based on predict_log_proba value of 5 trained-validated-models ===
defaultdict(<class 'list'>, {'18219': [['GX004-93-7097963', -0.4661388896041218, 0.0], ['GX016-32-14546147', -0.5352620960133454, 0.0], ['GX020-25-8391882', -1.219694227347485, 1.0], ['GX026-03-13004845', -1.5765487799035125, 0.0], ['GX025-94-0531672', -1.6290414816581456, 0.0], ['GX268-53-13016636', -1.8779070764562535, 0.0], ['GX048-02-13747475', -1.9910671286674029, 0.0], ['GX010-40-4497720', -2.022578133972534, 0.0]], '18230': [['GX230-44-5924225', -0.6448860115318494, 1.0], ['GX234-25-8280949', -0.714984754806431, 1.0], ['GX111-60-4484292', -0.7657723232995991, 1.0], ['GX001-00-5105044', -0.8013086706383749, 1.0], ['GX005-40-1622912', -0.8200425461515904, 1.0], ['GX200-73-5240168', -0.9130894584871828, 1.0], ['GX061-04-16698930', -0.930850311705776, 1.0], ['GX027-23-15133882', -0.9535496148453496, 1.0], ['GX004-53-15843752', -1.0240246454627033, 1.0], ['GX236-92-10964728', -1.0302541852399296, 

add your results and discussion here
Point-wise Learning to Rank framework. 
1. You could train a binary classification model like SVM or logistic regression on the train file. In this case, 0 is treated as negative (irrelevant) sample and 1, 2 are treated as positive (relevant) sample.
2. You apply the already trained model to predict the scores for documents on test file.
3. Order the documents based on the scores.

[ANS]
1. Firstly, parsed the x, y, q_id from the MQ2008 dataset. And transform the scope of y from {0,1,2}->{0,1}
2. Secondly, imported the SVM model on train.txt of Fold1~Fold5, then validated it on validate.txt to acquire the best set of HyperParameters, which is C=1, gamma=0.5, kernel='rbf'. (Validated by adopting GridSearchCV)
3. Feedback the  HyperParameters set to each SVM model, then train the model once on train.txt of Fold1~Fold5.
4. Rank the (q,d) based on the re-trained model.


## Part 2.2: NDCG

Based on your prediction file (results could be ranked by scores in the prediction file) and ground-truth (i.e., 0,1,2) in the test file, calculate NDCG for each query. Report average NDCG for all queries in the five-fold cross validation.

For NDCG, please bulid your own function rather then using any package.
<br><br>[NDCG WIKI](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)
<br>Christopher D. Manning, Prabhakar Raghavan, and Hinrich Schutze, "Introduction to Information Retrieval"
<br>[NDCG](https://nlp.stanford.edu/IR-book/pdf/08eval.pdf) (Chap 8.4)
 

In [0]:
# your code here
import math
import operator

def Calculate_DCG_10(relevances, rank=10):
    # Same formula for both DCG and iDCG at top10
    num_relevances = len(relevances)
    if num_relevances == 0:
        return 0.0
    rank = min(rank, num_relevances)
    relevances = np.asarray(relevances)[:rank]
    
    idx = 1
    dcg = 0.0
    for rel in relevances:
        dcg = dcg + (2**rel-1)/math.log2(1 + idx)
        idx = idx+1
    return dcg
 
def Calculate_NDCG_10(dcg, idcg):
    if idcg == 0:
        return 0.0
    return dcg / idcg

def Calculate_NDCG_Fold (unique_query, Ranked_of_q_doc, Ground_Truth_q_doc):
    # unique_query, Ranked_of_q_doc, Ground_Truth_q_doc (1~5)
    # Ranked_of_q_doc: [ doc_id[idx], log_prob[idx], y_test[idx] ]
    # dcg, idcg, ndcg of specific query
    ndcg = [0] * len(unique_query)
    idx = 0
    
    for query in unique_query:
        # Fetch the "relevance" of Ranked & Truth
        tmp1 = Ranked_of_q_doc[str(query)]
        tmp2 = Ground_Truth_q_doc[str(query)]
        Ranked_rel = []
        Truth_rel = []
        for i in tmp1:
            Ranked_rel.append(i[2])
        for i in tmp2:
            Truth_rel.append(i[2])
        # Calculate the dcg, idcg, ndcg of specific query
        dcg = Calculate_DCG_10(Ranked_rel)
        idcg = Calculate_DCG_10(Truth_rel)
        ndcg[idx] = Calculate_NDCG_10(dcg, idcg)
        idx = idx + 1
    ndcg_mean = sum(ndcg)/len(ndcg)
    return ndcg_mean

############# Calculate NDCG for 5 Folds based on unique_query, Ranked_of_q_doc, Ground_Truth_q_doc #############
ndcg_mean1 = Calculate_NDCG_Fold (unique_query1, Ranked_of_q_doc1, Ground_Truth_q_doc1)
ndcg_mean2 = Calculate_NDCG_Fold (unique_query2, Ranked_of_q_doc2, Ground_Truth_q_doc2)
ndcg_mean3 = Calculate_NDCG_Fold (unique_query3, Ranked_of_q_doc3, Ground_Truth_q_doc3)
ndcg_mean4 = Calculate_NDCG_Fold (unique_query4, Ranked_of_q_doc4, Ground_Truth_q_doc4)
ndcg_mean5 = Calculate_NDCG_Fold (unique_query5, Ranked_of_q_doc5, Ground_Truth_q_doc5)

print ("Fold 1 NDCG@10:  " + str(ndcg_mean1) )
print ("Fold 2 NDCG@10:  " + str(ndcg_mean2) )
print ("Fold 3 NDCG@10:  " + str(ndcg_mean3) )
print ("Fold 4 NDCG@10:  " + str(ndcg_mean4) )
print ("Fold 5 NDCG@10:  " + str(ndcg_mean5) )

print ("Average NDCG@10 of model clf4: " + str( (ndcg_mean1+ndcg_mean2+ndcg_mean3+ndcg_mean4+ndcg_mean5)/5.0 ))


Fold 1 NDCG@10:  0.4348077505364751
Fold 2 NDCG@10:  0.4235916688202801
Fold 3 NDCG@10:  0.4475543006416738
Fold 4 NDCG@10:  0.5217009197346872
Fold 5 NDCG@10:  0.5111391458062383
Average NDCG@10 of model clf4: 0.46775875710787085
